In [15]:
import telebot
from telebot.types import Message
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from time import sleep

#get API token and start bot
f=open("API_TOKEN","r")
lines=f.readlines()
API_TOKEN = lines[0]
f.close()

f=open("SBER_AUTH","r")
lines=f.readlines()
sber = lines[0]
f.close()

llm = GigaChat(credentials=sber, verify_ssl_certs=False)
template = '''
Ты — эксперт по фасилитации. Твоя задача отвечать \
на вопросы по проведению и подготовке к фасилитации. Ты говоришь дружелюбно, уверенно, четко. \
Объясняешь даже сложные концепции простым языком. Приводишь примеры, где это уместно. \
Не усложняешь без необходимости, но всегда держишь высокий уровень профессионализма.\
Если задают вопросы по другим темам, говори что это тебе не интересно. Не отвечай на вопросы не про фасилитацию! Это важно!
\n\nТекущий разговор:\n{history}\nHuman: {input}\nAI:
'''
#убрал пока 'красочно и подробно'
conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())
conversation.prompt.template = template

bot = telebot.TeleBot(API_TOKEN)

# Keep track of user data: user_id -> {'prompt_msg_id': int, 'session_text': str, 'accepted_msg_id': int}
user_sessions = {}
user_conversations = {}

@bot.message_handler(commands=['start'])
def handle_start(message: Message):
    prompt = bot.send_message(message.chat.id, "Write the details of a session you want to facilitate as a reply if you have a theoretical question use /coach")
    user_sessions[message.from_user.id] = {'prompt_msg_id': prompt.message_id}

@bot.message_handler(commands=['coach'])
def handle_coach(message: Message):
    user_id = message.chat.id

    # Проверка, существует ли уже ConversationBufferMemory для данного пользователя
    if user_id not in user_conversations:
        user_conversations[user_id] = ConversationBufferMemory()

    # Обновление конфигурации ConversationChain для текущего пользователя
    conversation.memory = user_conversations[user_id]

    # Получение и отправка ответа через GigaChat
    response = conversation.predict(input=message.text)
    bot.send_message(user_id, conversation.memory.chat_memory.messages[-1].content, parse_mode='Markdown')
    sleep(2)

@bot.message_handler(func=lambda m: m.reply_to_message is not None and not m.text.startswith("/"))
def handle_session_reply(message: Message):
    user_id = message.from_user.id
    replied_to = message.reply_to_message

    session_info = user_sessions.get(user_id)
    if session_info and replied_to.message_id == session_info.get('prompt_msg_id'):
        # Save the user's session message
        session_info['session_text'] = message.text
        plan = conversation.predict(input='Составь фасилитационный план с таймингами и учётом цели и количества участников встречи по следующему запросу ' + 
                                                         session_info.get('session_text', '[No session text found]'))
        session_info['plan'] = plan
        accepted_msg = bot.reply_to(message, plan + '\n\n Reply to this message with /transcript <transcript> so we can analyze your meeting',parse_mode='Markdown')
        session_info['accepted_msg_id'] = accepted_msg.message_id

@bot.message_handler(commands=['transcript'])
def handle_transcript(message: Message):
    user_id = message.from_user.id
    replied_to = message.reply_to_message

    session_info = user_sessions.get(user_id)
    if session_info and replied_to and replied_to.message_id == session_info.get('accepted_msg_id'):
        # Get the <text> from the /transcript command
        parts = message.text.split(' ', 1)
        if len(parts) < 2:
            bot.reply_to(message, "Please provide text after /transcript.")
            return

        transcript_text = parts[1]
        plan = session_info.get('plan', '[No session text found]')
        reply = conversation.predict(input='Дай советы по улучшению проведённой встречи на основе её расшифровки: \n"' + 
                                     transcript_text + 
                                     '"\n\n и фасилитационного плана:' + 
                                     plan)
        bot.reply_to(message, reply,parse_mode='Markdown')

bot.infinity_polling()




> Entering new ConversationChain chain...
Prompt after formatting:

Ты — эксперт по фасилитации. Твоя задача отвечать на вопросы по проведению и подготовке к фасилитации. Ты говоришь дружелюбно, уверенно, четко. Объясняешь даже сложные концепции простым языком. Приводишь примеры, где это уместно. Не усложняешь без необходимости, но всегда держишь высокий уровень профессионализма.Если задают вопросы по другим темам, говори что это тебе не интересно. Не отвечай на вопросы не про фасилитацию! Это важно!


Текущий разговор:

Human: /coach что такое ретро
AI:


> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:

Ты — эксперт по фасилитации. Твоя задача отвечать на вопросы по проведению и подготовке к фасилитации. Ты говоришь дружелюбно, уверенно, четко. Объясняешь даже сложные концепции простым языком. Приводишь примеры, где это уместно. Не усложняешь без необходимости, но всегда держишь высокий уровень профессионализма.Если задают вопросы по другим тем

2025-04-29 21:33:02,649 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-04-29 21:33:02,650 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
